### Main Class for the API Server
- It is an AI server that can provide recommendation for categories and super latents.
- Uses Python 3 and tensorflow 2.4
- Written in Jan 2021
- It has no dependencies, and can be run directly.
- There two end points:
 - POST /initial
   - body: json format of survey, mainly the latent 
   - return: json format of suggested category (list) and super latent with the same order as latent from the request (list)
```
POST Body:
{
   "clientName":"Office Max",
   "name":"OfficeMax In-Store Pick-Up v2",
   "surveyType":"PREDCSAT_NPS",
   "channelType":"STORE",
   "latents":[
      {
         "name":"Purchase Online",
         "type":"FUTURE_BEHAVIOR"
      },
      {
         "name":"Recommend",
         "type":"FUTURE_BEHAVIOR"
      },
      {
         "name":"Satisfaction",
         "type":"SATISFACTION"
      },
      {
         "name":"Store Employees",
         "type":"ELEMENT"
      },
      {
         "name":"Price",
         "type":"ELEMENT"
      },
      {
         "name":"Order Quality",
         "type":"ELEMENT"
      },
      {
         "name":"Ordering Process",
         "type":"ELEMENT"
      },
      {
         "name":"Order Pickup",
         "type":"ELEMENT"
      },
      {
         "name":"Email Communications",
         "type":"ELEMENT"
      },
      {
         "name":"Recommend Store Pickup",
         "type":"FUTURE_BEHAVIOR"
      },
      {
         "name":"Use Store Pickup",
         "type":"FUTURE_BEHAVIOR"
      },
      {
         "name":"Purchase Offline",
         "type":"FUTURE_BEHAVIOR"
      }
   ]
}
```
```
Response:
{
    "categories": [
        "Multi-Channel Experience",
        "Multi-Channel Shopping Experience"
    ],
    "slatents": [
        "In-Channel Purchase",
        "Recommend",
        "Satisfaction",
        "Store Employees",
        "Price",
        "Order Quality",
        "Ordering Process",
        "Order Pick-up",
        "Email Communication",
        "Recommend Store Pickup",
        "Use Store Pick-up",
        "Other-Channel Purchase"
    ]
}
```
 - POST /cats
   - body: existing category names (list)
   - return: top 10 category suggested (list)
```
Body:
["Fulfillment"]
```
```
Response:
[
    "Retail - Fulfillment",
    "Retail Specialty Retailers - Fulfillment",
    "Retail Buy Online, Ship to Home - Fulfillment",
    "Multi-Channel Experience",
    "Multi-Channel Shopping Experience",
    "Retail Buy Online, Pickup in Store - Fulfillment",
    "Retail Apparel - Fulfillment",
    "Multi-Channel Membership Experience",
    "Manufacturers - Fulfillment",
    "Retail Hardware and Home Centers - Fulfillment"
]
```

In [ ]:
# !pip install falcon

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import falcon
import json
import re

In [ ]:
class CategoryManager:
    def normalize(self,sentence : str):
        return re.sub('[^0-9a-zA-Z_\\.]+', ' ', sentence)

    # return a list of name of categories
    def predict(self, json_data : dict):
        data = self.normalize(json.dumps(json_data))
        p = self.model.predict([data])[0] # we only predict one record
        r = []
        for idx, v in enumerate ((p > .5).tolist()):
            if v:
                r.append(self.cats[idx])
        return r

    # return a list of name of categories
    def recommend(self, cat_name : str):
        if(cat_name not in self.cats_full):
            return None
        cat_c = self.cats_full.index(cat_name)
        cat_ref_count = [0] * len(self.matrix)
        for idx, v in enumerate (self.matrix):
            if v[cat_c] == 1 :
                for ridx, rv in enumerate (v):
                    cat_ref_count[ridx] += 1 if rv == 1 else 0
        recommond = {} # dict {name: count}
        for idx, v in enumerate (cat_ref_count):
            if v > 0 and idx != cat_c:
                recommond[self.cats_full[idx]] = v
        return dict(sorted(recommond.items(), key=lambda item: item[1], reverse=True))
        

    def __init__(self):
        raw_data = pd.read_csv('data/S_SURVEY_FULL.csv')
        model_weights_path = 'model/0'

        # prepare the model data
        _cat_list = list()
        _min_num = len(raw_data) * .05

        for c in list(raw_data.columns)[1:(len(raw_data.columns)-1)]:
            if(raw_data[[c]].sum().tolist()[0] > _min_num):
                _cat_list.append(c)
                
        self.cats = _cat_list
        self.model = tf.keras.models.load_model(model_weights_path)

        # prepare the category recommendation data
        df = raw_data[raw_data.columns[1:(len(raw_data.columns)-1)]]
        self.matrix = np.array(df)
        self.cats_full = df.columns.tolist()


In [ ]:
class SuperLatentManager:
    def __init__(self):
        raw_data = pd.read_csv('data/super_latent.csv',header=None)
        raw_data['index'] = raw_data[0] + '__' + raw_data[1].astype(str)
        df = raw_data.set_index('index').drop([0,1], axis=1)
        df.rename(columns={2:'value'})
        self.dict = df # <name__type> as index, super latent as value pandas dataFrame

    def latent_type(self,l_type : str):
        return 3 if 'FUTURE_BEHAVIOR' == l_type else 2 if 'SATISFACTION' == l_type else 1
        
    def lookup(self, latent : str, l_type : str):
        return self.lookup_i(latent, self.latent_type(l_type))
        
    def lookup_i(self, latent : str, type_int : int):
        value_idx = latent + '__' + str(type_int)
        return self.dict.loc[value_idx].values[0] if (value_idx in self.dict.index) else None

In [ ]:
class App:
    def __init__(self, cat_mgr : CategoryManager, slatent_mgr : SuperLatentManager):
        self.api = falcon.API()
        self.api.add_route('/inital', self.InitialHandler(cat_mgr,slatent_mgr))
        self.api.add_route('/cats', self.CategoryHandler(cat_mgr))
        
    class InitialHandler:
        def __init__(self, c_m : CategoryManager, s_m : SuperLatentManager):
            self.c_m = c_m
            self.s_m = s_m

        def on_post(self, req, resp):
            """Handles POST requests"""
            resp.status = falcon.HTTP_200  # This is the default status
            text = req.stream.read().decode('utf-8') #req.get_param() #self.predict()
            json_data = json.loads(text)
            result = {'categories': self.c_m.predict(json_data), 
                      'slatents':[self.s_m.lookup(d['name'].strip(),d['type']) for d in json_data['latents']]}
            resp.body = json.dumps(result)

    class CategoryHandler:
        def __init__(self, c_m : CategoryManager):
            self.c_m = c_m

        def on_post(self, req, resp):
            """Handles POST requests"""
            resp.status = falcon.HTTP_200  # This is the default status
            text = req.stream.read().decode('utf-8') #req.get_param() #self.predict()
            json_data = json.loads(text)
            dic = dict()
            for elem in [self.c_m.recommend(x.strip()) for x in json_data]:
                for key, value in elem.items():
                    if(key in dic):
                        dic[key] += value
                    else:
                        dic[key] = value
            for x in json_data:
                dic.pop(x, None) 
            top10 = sorted(dic.items(), key=lambda x: x[1], reverse=True)[:10] # sorted[name, count]
            resp.body = json.dumps([x[0] for x in top10])


In [ ]:
app = App(CategoryManager(), SuperLatentManager()).api

In [ ]:
# inside docker run
# jupyter nbconvert --to python App.ipynb ; gunicorn App:app --bind 0.0.0.0:8000